# IESB - Miner II - Aula 04 - Random Forest

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# Carregando os dados
df = pd.read_csv('/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv')

df.shape

In [ ]:
# Olhando os dados
df.head().T

In [ ]:
# Olhando os dados aleatoriamente
df.sample(5).T

In [ ]:
# Verificando os tipos dos dados e os tamanhos
df.info()

In [ ]:
# Convertendo a coluna TotalCharges para float
df['TotalCharges'] = df['TotalCharges'].astype(float)

In [ ]:
# Vamos identificar os valores em branco na coluna TotalCharges
df[df['TotalCharges'].str.contains(' ')]

In [ ]:
# Para corrigir o TotalCharges vamos trocar o espaço em branco
# pelo valor ZERO e forçar novamente a conversão
df['TotalCharges'] = df['TotalCharges'].str.replace(' ', '0').astype(float)

# df['TotalCharges'] = df['TotalCharges'].str.strip().replace('', '0').astype(float)

In [ ]:
# Verificando novamente os tipos de dados
df.info()

In [ ]:
# Copiando o dataframe
df2 = df.copy()

In [ ]:
# Criando variaveis dummy para gender
pd.get_dummies(df['gender'])

In [ ]:
# Dummy da coluna PaymentMethod
pd.get_dummies(df['PaymentMethod'])

In [ ]:
# Criando dummys para todas as colunas
df = pd.get_dummies(df, columns=['gender', 'Partner', 'Dependents',
                                'PhoneService', 'MultipleLines',
                                'InternetService', 'OnlineSecurity',
                                'OnlineBackup', 'DeviceProtection',
                                'TechSupport', 'StreamingTV',
                                'StreamingMovies', 'Contract',
                                'PaperlessBilling', 'PaymentMethod'])

In [ ]:
df.shape

In [ ]:
# olhando os dados
df.head().T

In [ ]:
# Vrificando novamente os tipos dos dados
df.info()

In [ ]:
# Treinamento do modelo

# Separando o dataframe

# Importando o train_test_split
from sklearn.model_selection import train_test_split

# Separando treino e teste
train, test = train_test_split(df, test_size=0.20, random_state=42)

# Separando treino e validação
train, valid = train_test_split(train, test_size=0.20, random_state=42)

train.shape, valid.shape, test.shape

In [ ]:
# definindo colunas de entrada
feats = [c for c in df.columns if c not in ['customerID', 'Churn']]

feats

In [ ]:
# treinar o modelo

# Importando o modelo
from sklearn.ensemble import RandomForestClassifier

# Instanciar o modelo
rf = RandomForestClassifier(n_estimators=200, random_state=42)

In [ ]:
# treinar o modelo
rf.fit(train[feats], train['Churn'])

In [ ]:
# Prevendo os dados de validação
preds_val = rf.predict(valid[feats])

preds_val

In [ ]:
# Avaliando o desempenho do modelo

# Importando a metrica
from sklearn.metrics import accuracy_score

In [ ]:
# Acurácia das previsões de validação
accuracy_score(valid['Churn'], preds_val)

In [ ]:
# Medindo a acurácia nos dados de teste
preds_test = rf.predict(test[feats])

accuracy_score(test['Churn'], preds_test)

In [ ]:
# Olhando a coluna Churn do dataframe completo
df['Churn'].value_counts(normalize=True)

In [ ]:
# Olhando a coluna Churn do dataframe completo
df['Churn'].value_counts()

In [ ]:
df2.info()

In [ ]:
# Tipo category no pandas
df2['gender'].astype('category').cat.categories

In [ ]:
df2['gender'].astype('category').cat.codes

In [ ]:
df2['PaymentMethod'].astype('category').cat.codes

In [ ]:
df2['PaymentMethod'].astype('category').cat.categories

In [ ]:
# Convertendo as colunas categórias em colunas numéricas
for col in df2.columns:
    if df2[col].dtype == 'object':
        df2[col] = df2[col].astype('category').cat.codes

In [ ]:
# Verificando novamente os tipos
df2.info()

In [ ]:
# Separando o dataframe em train, valid e test

# Primeiro, train e test
train, test = train_test_split(df2, test_size=0.2, random_state=42)

# Depois, train e valid
train, valid = train_test_split(train, test_size=0.2, random_state=42)

train.shape, valid.shape, test.shape

In [ ]:
# Colunas a serem usadas para treino
feats2 = [c for c in df2.columns if c not in ['customerID', 'Churn']]

In [ ]:
# Instanciando o modelo
rf2 = RandomForestClassifier(n_estimators=200, random_state=42)

# Treinando o modelo
rf2.fit(train[feats2], train['Churn'])

In [ ]:
# previsões para os dados de validação
preds_val = rf2.predict(valid[feats2])

# Verificando a acurácia
accuracy_score(valid['Churn'], preds_val)

In [ ]:
# Previsões para os dados de teste
preds_test = rf2.predict(test[feats2])

# Verificando a acurácia
accuracy_score(test['Churn'], preds_test)

In [ ]:
# Avaliando a importancia de cada coluna (cada variável de entrada)
pd.Series(rf2.feature_importances_, index=feats2).sort_values().plot.barh()

In [ ]:
# Avaliando a importancia de cada coluna no RF1 (cada variável de entrada)
pd.Series(rf.feature_importances_, index=feats).sort_values().plot.barh()

In [ ]:
# importando a bilbioteca para plotar o gráfico de Matriz de Confusão
import scikitplot as skplt

In [ ]:
# Matriz de Confusão - Dados de Validação
skplt.metrics.plot_confusion_matrix(valid['Churn'], preds_val)